In [37]:
import requests
import json
import pandas as pd
from matplotlib import pyplot as plt

In [21]:
url="https://data.cms.gov/provider-data/api/1/metastore/schemas/dataset/items/4pq5-n9py?show-reference-ids=false"

In [22]:
print(requests.get(url))

<Response [200]>


In [57]:
print(requests.get(url).json())

{'accessLevel': 'public', 'landingPage': 'https://data.cms.gov/provider-data/dataset/4pq5-n9py', 'bureauCode': ['009:38'], 'issued': '2023-08-01', '@type': 'dcat:Dataset', 'modified': '2024-04-01', 'released': '2024-04-24', 'keyword': [{'identifier': 'd03021cc-311c-5181-8eb6-7ccf7e76fc53', 'data': 'General Information'}, {'identifier': '203355e5-2454-505d-b356-08de15938ab9', 'data': 'Address'}, {'identifier': 'c59e48c7-3cc5-5a09-adec-a49493db72f6', 'data': 'Location'}, {'identifier': '70a212e5-bf1b-5cad-9ecd-150dab2bacf9', 'data': 'Ratings'}, {'identifier': '107b8826-e90b-5db3-ac30-ecdb99c73db6', 'data': 'Beds'}, {'identifier': '5692b1a6-1c82-55c4-9698-c7441733b985', 'data': 'Quality'}, {'identifier': 'a6f7b7c6-d26e-5f59-8a5b-c151bacb9907', 'data': 'Staffing'}, {'identifier': '35df7004-e437-5a8a-a96b-1bda5456ccca', 'data': 'Penalties'}], 'contactPoint': {'@type': 'vcard:Contact', 'fn': 'Nursing homes including rehab services', 'hasEmail': 'mailto:BetterCare@cms.hhs.gov'}, 'publisher': 

In [58]:
response = requests.get(url).json()
print(json.dumps(response, indent=4, sort_keys=True))

{
    "%modified": "2024-04-12T16:05:17-0400",
    "@type": "dcat:Dataset",
    "accessLevel": "public",
    "bureauCode": [
        "009:38"
    ],
    "contactPoint": {
        "@type": "vcard:Contact",
        "fn": "Nursing homes including rehab services",
        "hasEmail": "mailto:BetterCare@cms.hhs.gov"
    },
    "description": "General information on currently active nursing homes, including number of certified beds, quality measure scores, staffing and other information used in the Five-Star Rating System. Data are presented as one row per nursing home.",
    "distribution": [
        {
            "data": {
                "%Ref:downloadURL": [
                    {
                        "data": {
                            "checksum": null,
                            "filePath": "s3://pdc-s3-nhc-data/NH_ProviderInfo_Apr2024.csv",
                            "identifier": "5eba4f43478aeb4d0ab8d071e1e4655d",
                            "mimeType": "text/csv",
           

In [12]:
response = requests.get(url).json()
print(json.dumps(response["distribution"][0]["data"]["downloadURL"], indent=4, sort_keys=True))

"https://data.cms.gov/provider-data/sites/default/files/resources/5eba4f43478aeb4d0ab8d071e1e4655d_1712952317/NH_ProviderInfo_Apr2024.csv"


In [13]:
nursinghomedataURL=response["distribution"][0]["data"]["downloadURL"], indent=4, sort_keys=True))

In [60]:
print(nursinghomedataURL)

https://data.cms.gov/provider-data/sites/default/files/resources/5eba4f43478aeb4d0ab8d071e1e4655d_1712952317/NH_ProviderInfo_Apr2024.csv


In [25]:
# pd.read_csv(nursinghomedataURL).sample(5)
nursinghomedata=pd.read_csv(nursinghomedataURL)
nursinghomedata.sample(5)

,CMS Certification Number (CCN),Provider Name,Provider Address,City/Town,State,ZIP Code,Telephone Number,Provider SSA County Code,County/Parish,Ownership Type,...,Number of Citations from Infection Control Inspections,Number of Fines,Total Amount of Fines in Dollars,Number of Payment Denials,Total Number of Penalties,Location,Latitude,Longitude,Geocoding Footnote,Processing Date
2776,11A200,GRACEWOOD NSG FACILITY(UNIT 9),"100 MYRTLE BLVD., EAST CENTRAL REG HOSP",GRACEWOOD,GA,30812,7067902012,840,Richmond,Government - State,...,NaN,0,0.00,0,0,"100 MYRTLE BLVD., EAST CENTRAL REG HOSP,GRACEW...",33.3739,-82.026,22.0,2024-04-01
7571,285116,CLARKSON COMMUNITY CARE CENTER INC,212 SUNRISE DRIVE,CLARKSON,NE,68629,4028923494,180,Colfax,Non profit - Corporation,...,NaN,1,15592.85,1,2,"212 SUNRISE DRIVE,CLARKSON,NE,68629",41.7264,-97.130,NaN,2024-04-01
10639,385188,INDEPENDENCE HEALTH & REHABILITATION CENTER,1525 MONMOUTH STREET,INDEPENDENCE,OR,97351,5038380001,260,Polk,For profit - Limited Liability company,...,0.0,0,0.00,0,0,"1525 MONMOUTH STREET,INDEPENDENCE,OR,97351",44.8512,-123.211,NaN,2024-04-01
1436,065407,ADVANCED HEALTH CARE OF COLORADO SPRINGS,55 S PARKSIDE DR,COLORADO SPRINGS,CO,80910,7194184500,200,El Paso,For profit - Limited Liability company,...,NaN,1,650.00,0,1,"55 S PARKSIDE DR,COLORADO SPRINGS,CO,80910",38.8313,-104.785,NaN,2024-04-01
7538,285065,GOLD CREST RETIREMENT CENTER,200 LEVI LANE,ADAMS,NE,68301,4029887115,330,Gage,Non profit - Corporation,...,2.0,2,13650.00,0,2,"200 LEVI LANE,ADAMS,NE,68301",40.4541,-96.521,NaN,2024-04-01


In [35]:
# nursinghomedataREDUCED=nursinghomedata.drop(columns=['CMS Certification Number (CCN)', 'Provider SSA County Code', 'Geocoding Footnote', 'Processing Date'])
nursinghomedataREDUCED=nursinghomedata[[
        'Provider Name', 
        'City/Town',
        'State', 
        'Ownership Type', 
        'Number of Certified Beds', 
        'Average Number of Residents per Day', 
        'Number of Facility Reported Incidents', 
        'Number of Substantiated Complaints', 
        'Number of Citations from Infection Control Inspections', 
        'Number of Fines', 
        'Total Amount of Fines in Dollars',
        'Abuse Icon']]
    

nursinghomedataREDUCED.sample(25)

,Provider Name,City/Town,State,Ownership Type,Number of Certified Beds,Average Number of Residents per Day,Number of Facility Reported Incidents,Number of Substantiated Complaints,Number of Citations from Infection Control Inspections,Number of Fines,Total Amount of Fines in Dollars,Abuse Icon
11801,LIFE CARE CENTER OF CROSSVILLE,CROSSVILLE,TN,For profit - Corporation,122,78.2,0,6,3.0,1,7003.75,N
2402,DE LUNA HEALTH AND REHABILITATION CENTER,PENSACOLA,FL,For profit - Limited Liability company,90,79.6,0,0,0.0,0,0.00,N
8996,BETHESDA HEALTH CARE FACILITY,EASTOVER,NC,For profit - Corporation,85,70.6,0,0,NaN,1,655.01,N
5149,J. MICHAEL MORROW MEMORIAL NURSING HOME,ARNAUDVILLE,LA,For profit - Partnership,175,149.4,1,0,0.0,1,4194.45,N
10866,BROAD ACRES HEALTH AND REHABILITATION,WELLSBORO,PA,Non profit - Corporation,120,79.2,0,6,NaN,1,650.00,N


In [46]:
nursinghomedata.columns

Index(['CMS Certification Number (CCN)', 'Provider Name', 'Provider Address',
       'City/Town', 'State', 'ZIP Code', 'Telephone Number',
       'Provider SSA County Code', 'County/Parish', 'Ownership Type',
       'Number of Certified Beds', 'Average Number of Residents per Day',
       'Average Number of Residents per Day Footnote', 'Provider Type',
       'Provider Resides in Hospital', 'Legal Business Name',
       'Date First Approved to Provide Medicare and Medicaid Services',
       'Affiliated Entity Name', 'Affiliated Entity ID',
       'Continuing Care Retirement Community', 'Special Focus Status',
       'Abuse Icon', 'Most Recent Health Inspection More Than 2 Years Ago',
       'Provider Changed Ownership in Last 12 Months',
       'With a Resident and Family Council',
       'Automatic Sprinkler Systems in All Required Areas', 'Overall Rating',
       'Overall Rating Footnote', 'Health Inspection Rating',
       'Health Inspection Rating Footnote', 'QM Rating', 'QM Rating